In [0]:
import cv2
import numpy as np
import os
from keras import Input
from keras import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.utils import plot_model
from keras.layers.core import Activation
from keras.layers.core import Dropout

def getImageVariable(path):
	classFolder=path
	# imagePath="/home/bharat/DL/assignment1/classes"
	imageList=[]
	cnt=0
	tempArry=np.zeros((96000,5),int)

	for length in range(2):
		for width in range(2):
			for angle in range(12):
				for color in range(2):

					for variation in range(1000):
	
						tempArry[cnt,:]=[cnt,length,width,angle,color]
						cnt+=1
	np.random.shuffle(tempArry)

	
	for i in range(96000):
		string="/"+str(tempArry[i][1])+"_"+str(tempArry[i][2])+"_"+str(tempArry[i][3])+"_"+str(tempArry[i][4])
		imgFolder=classFolder+string
		img=imgFolder+string+"_"+str(tempArry[i][0]%1000)+".jpeg"
		imageList.append(cv2.imread(img))

	return imageList,tempArry[:,1],tempArry[:,2],tempArry[:,3],tempArry[:,4]


imagePath="/content/home/classes" # jaha imagees ka folder h
a, b, c, d, e = getImageVariable(imagePath)

In [67]:
# from keras import Input
# from keras import Model
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import Conv2D
# from keras.layers.convolutional import MaxPooling2D
# from keras.layers.core import Dense
# from keras.layers import Flatten
# from keras.utils import plot_model

# Single input and four output(classification heads)
# Test and train split
# First output length wise 0-7 and 1-15
# second output width wise 0-1 and 1-3
# third output color wise 0-red and 1-blue
# fourth output angle wise (0-180, diff=15) - (0-11)



# remember to normalize image array by dividing 255
# try out batch normalization before and after activation

input_train = np.array(a, dtype=np.float32)
input_train /= 255.0
print(input_train.shape)
# # give names to all below - output1, ...
output1_train = np.array(b)
output2_train = np.array(c)
print(output2_train.shape)
output3_train = np.array(e)
output4_train = np.array(d)

splitSz = input_train.shape[0]
# splitSz = 60000
endd = splitSz
splitSz = int(0.7*splitSz)

input_train, input_test = input_train[:splitSz,:], input_train[splitSz:endd,:]
output1_train, output1_test = output1_train[:splitSz], output1_train[splitSz:endd]
output2_train, output2_test = output2_train[:splitSz], output2_train[splitSz:endd]
output3_train, output3_test = output3_train[:splitSz], output3_train[splitSz:endd]
output4_train, output4_test = output4_train[:splitSz], output4_train[splitSz:endd]

(96000, 28, 28, 3)
(96000,)


In [68]:

inp = Input(shape=(28,28,3))

# Conv -> Conv -> Pool
x = Conv2D(32, (3,3), padding="same")(inp)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(32, (3,3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2,2))(x)


# Conv -> Conv -> Pool
x = Conv2D(64, (3,3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(64, (3,3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)

# above is the shared layer and our x now
# is the feature map.
# x = (7, 7, 64)

# out1 corresponds to the length clasification head
out1 = Dense(1024)(x)
out1 = Activation("relu")(out1)
out1 = Dropout(0.5)(out1)
# out1 = Dense(1024)(out1)
# out1 = Activation("relu")(out1)
# out1 = Dropout(0.3)(out1)
out1 = Dense(1)(out1)
out1 = Activation("sigmoid", name="out1")(out1)

# out2 corresponds to the width clasification head
out2 = Dense(1024)(x)
out2 = Activation("relu")(out2)
out2 = Dropout(0.5)(out2)
# out2 = Dense(1024)(out2)
# out2 = Activation("relu")(out2)
# out2 = Dropout(0.5)(out2)
out2 = Dense(1)(out2)
out2 = Activation("sigmoid", name="out2")(out2)

# out3 corresponds to the color clasification head
out3 = Dense(1024)(x)
out3 = Activation("relu")(out3)
out3 = Dropout(0.5)(out3)
# out3 = Dense(1024)(out3)
# out3 = Activation("relu")(out3)
# out3 = Dropout(0.5)(out3)
out3 = Dense(1)(out3)
out3 = Activation("sigmoid", name="out3")(out3)

# out4 corresponds 
out4 = Dense(1024)(x)
out4 = Activation("relu")(out4)
# out4 = Dropout(0.2)(out4)
# out4 = Dense(1024)(out4)
# out4 = Activation("relu")(out4)
# out4 = Dropout(0.2)(out4)
out4 = Dense(12)(out4)
out4 = Activation("softmax", name="out4")(out4)

# final model with one input and four classification heads
model = Model(inputs=[inp], outputs=[out1, out2, out3, out4])

model.summary()
plot_model(model, to_file='assign2b.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 28, 28, 32)   896         input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 28, 28, 32)   128         conv2d_56[0][0]                  
__________________________________________________________________________________________________
activation_145 (Activation)     (None, 28, 28, 32)   0           batch_normalization_56[0][0]     
__________________________________________________________________________________________________
conv2d_57 

In [69]:


model.compile(optimizer='adam', loss={'out1': 'binary_crossentropy', 'out2': 'binary_crossentropy', 'out3': 'binary_crossentropy', 'out4': 'sparse_categorical_crossentropy'}, metrics=['accuracy'])
model.fit([input_train], {'out1': output1_train, 'out2': output2_train, 'out3': output3_train, 'out4': output4_train}, epochs=5, batch_size=128)
# result = model.predict(input_test)

# accuracy
# print(result.shape)



Epoch 1/5
67200/67200 [==============================] - 763s 11ms/step - loss: 1.7752 - out1_loss: 0.0467 - out2_loss: 0.0776 - out3_loss: 0.0116 - out4_loss: 1.6394 - out1_acc: 0.9868 - out2_acc: 0.9803 - out3_acc: 0.9971 - out4_acc: 0.8258
Epoch 2/5
67200/67200 [==============================] - 753s 11ms/step - loss: 1.3875 - out1_loss: 0.0023 - out2_loss: 0.0037 - out3_loss: 0.0017 - out4_loss: 1.3798 - out1_acc: 0.9994 - out2_acc: 0.9987 - out3_acc: 0.9996 - out4_acc: 0.9061
Epoch 3/5
67200/67200 [==============================] - 749s 11ms/step - loss: 1.3523 - out1_loss: 0.0053 - out2_loss: 4.2969e-04 - out3_loss: 0.0019 - out4_loss: 1.3448 - out1_acc: 0.9985 - out2_acc: 0.9998 - out3_acc: 0.9997 - out4_acc: 0.9162
Epoch 4/5
67200/67200 [==============================] - 750s 11ms/step - loss: 1.3858 - out1_loss: 0.0072 - out2_loss: 0.0065 - out3_loss: 0.0034 - out4_loss: 1.3686 - out1_acc: 0.9985 - out2_acc: 0.9986 - out3_acc: 0.9996 - out4_acc: 0.9091
Epoch 5/5
67200/67200 [=

In [70]:
print(input_test.shape)
result = model.predict(input_test)
score = model.evaluate(input_test, {'out1': output1_test, 'out2': output2_test, 'out3': output3_test, 'out4': output4_test})
print(model.metrics_names)
# print(result[3])


(28800, 28, 28, 3)
28800/28800 [==============================] - 108s 4ms/step
['loss', 'out1_loss', 'out2_loss', 'out3_loss', 'out4_loss', 'out1_acc', 'out2_acc', 'out3_acc', 'out4_acc']


In [72]:
# resultSave = np.array((result))
print(score)

[1.6970876247375175, 0.0892230858825964, 0.0008753855714013131, 1.2236573836964401e-07, 1.6069890294734148, 0.9849305555555555, 0.9996527777777777, 1.0, 0.8569791666666666]
